In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from corpus_reading import read_dump

chords_tsv = 'data/chord_list.tsv'
notes_tsv = 'data/note_list.tsv'
measures_tsv = 'data/measure_list.tsv'
files_tsv = 'data/file_list.tsv'

chords_df = read_dump(chords_tsv)
notes_df = read_dump(notes_tsv, index_col=[0,1,2])
measures_df = read_dump(measures_tsv)
files_df = read_dump(files_tsv, index_col=0)

# Bugfixes
measures_df.loc[(685, 487), 'next'][0] = 488

In [ ]:
files_df

In [ ]:
chords_df

In [ ]:
measures_df

In [ ]:
notes_df

In [ ]:
import harmonic_inference_data as hid
import importlib
importlib.reload(hid)

train_dataset, valid_dataset, test_dataset = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='811split'
)

In [ ]:
import harmonic_inference_models as him
import harmonic_utils
import importlib
importlib.reload(him)

model = him.MusicScoreModel(len(train_dataset[0]['notes'][0]), len(harmonic_utils.CHORD_TYPES) * 12, dropout=0.2)

In [ ]:
import model_trainer
import importlib
importlib.reload(model_trainer)

from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

optimizer = Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001)
criterion = CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
schedule_var = 'valid_loss'

trainer = model_trainer.ModelTrainer(model, train_dataset=train_dataset, valid_dataset=valid_dataset,
                                     test_dataset=test_dataset, seed=0, num_epochs=100, early_stopping=20,
                                     optimizer=optimizer, scheduler=scheduler, schedule_var=schedule_var,
                                     criterion=criterion,
                                     log_every=1, log_file=None,
                                     save_every=10, save_dir='checkpoints', save_prefix='checkpoint',
                                     resume=None)

In [ ]:
trainer.train()

In [ ]:
trainer = model_trainer.ModelTrainer(model, train_dataset=train_dataset, valid_dataset=valid_dataset,
                                     test_dataset=test_dataset, seed=0, num_epochs=100, early_stopping=20,
                                     optimizer=optimizer, scheduler=scheduler, schedule_var=schedule_var,
                                     criterion=criterion,
                                     log_every=1, log_file=None,
                                     save_every=10, save_dir='checkpoints', save_prefix='checkpoint',
                                     resume='checkpoints/best.pth.tar')
loss, acc, outputs, labels = trainer.evaluate()

In [ ]:
print(loss, acc)

In [ ]:
import harmonic_utils

index = 0
label_strings = harmonic_utils.get_one_hot_labels()

print(f'Correct: {labels[index]} ({label_strings[labels[index]]})')
print(f'Guessed: {outputs[index].argmax()}  ({label_strings[outputs[index].argmax()]})')
print(f'Full: {outputs[index]}')
print(f'Labels: {label_strings}')

In [ ]:
conf_mat = np.zeros((len(label_strings), len(label_strings)))

for target, out in zip(labels, outputs):
    conf_mat[target, out.argmax()] += 1
    
# Normalize rows
conf_mat /= np.sum(conf_mat, axis=1, keepdims=True)
conf_mat = np.where(np.isnan(conf_mat), 0, conf_mat)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(30,30))
plt.imshow(conf_mat, interpolation='none')
plt.colorbar()
plt.xticks(ticks=list(range(len(label_strings))), labels=label_strings, rotation=90, fontsize=10)
plt.yticks(ticks=list(range(len(label_strings))), labels=label_strings, fontsize=10)
plt.show()